# Mapping NFL Drive Charts 

Reference: ArcGIS Python Recipes, "Mapping NFL Drive Charts." ArcPy Cafe, 2016. https://arcpy.wordpress.com/

[`nflgame`](https://pypi.org/project/nflgame/) is an API to retrieve and read NFL Game Center JSON data. It can work with real-time data. `nflgame` works by parsing the same JSON data that powers NFL.com's live GameCenter. The package comes pre-loaded with game data from every pre and regular season game from 2009 to present. Therefore, querying such data does not actually ping NFL.com. 

Example from [PyPI](https://pypi.org/project/nflgame/): Finding top 5 running backs by rushing yards in the first week of the 2013 season. 

In [ ]:
import nflgame

games = nflgame.games(2013,week=1)
players = nflgame.combine_game_stats(games)
for p in players.rushing().sort('rushing_yds').limit(5):
    msg = '%s %d carries for %d yards and %d TDs'
    print msg % (p, p.rushing_att, p.rushing_yds, p.rushing_tds)

Example from [PyPI](https://pypi.org/project/nflgame/): Finding top 5 passing plays in the same time period. 

In [ ]:
import nflgame
games = nflgame.games(2013, week = 1)
plays = nflgame.combine_plays(games)
for p in plays.sort('passing_yds').limit(5):
    print p


## ArcGIS

Working along the script in ArcGIS Python Recipe, we can create NFL drive charts in ArcGIS. A drive chart is a visual representation of a sequence of plays during a football game. 

This example looks at Superbowl XLIX with Seattle Seahawks vs. New England Patriots. 

In [ ]:
import os
import nflgame
import arcpy

In [ ]:
# Create football field feature class, assembled polygon features for the field, insert into featureclass

def build_football_field(output_gdb, output_feature_class):
    print('Creating football field')
    football_field_fields = ('SHAPE@', 'TEAM')
    fc = os.path.join(ooutput_gdb, output_feature_class)
    if not arcpy.Exists(os.path.join(output_gdb, output_feature_class)):
        arcpy.CreateFeatureclass_management(
            output_gdb, output_feature_class, "POLYGON", "#", "DISABLED",
            "DISABLED", arcpy.SpatialReference(3857))
        arcpy.AddField_management(fc,football_field_fields[1], "TEXT", field_length = 20)
        
    cursor = arcpy.da.InsertCursor(fc, football_field_fields)
    
    field = [(0, 533.3),(1000, 533.3), (1000, 0), (0, 0)]
    cursor.insertRow([field, ""])
    home_endzone = [(-100, 533.3), (0, 533.3), (0, 0), (-100, 0)]
    cursor.insertRow([home_endzone, "SEATTLE"])
    away_endzone = [(1000, 533.3), (1100, 533.3), (1100, 0), (1000, 0)]
    cursor.insertRow([away_endzone, "NEW ENGLAND"])
    # Field Dimensions are 100 yds x 53.33 yds, endzones are 10 yds deep. Length and width scaled by 10. 

In [ ]:
# Created yardline markers spaced at 10 yd intervals

def build_yard_lines(output_gdb, output_feature_class):
    print('Creating yard markers.')
    football_line_fields = ('SHAPE@', 'MARKER')
    fc = os.path.join(output_gdb, output_feature_class)
    if not arcpy.Exists(os.path.join(output_gdb, output_feature_class)):
        arcpy.CreateFeatureclass_management(
            output_gdb, output_feature_class, "POLYLINE", "#", "DISABLED", "DISABLED",
            arcpy.SpatialReference(3857))
        arcpy.AddField_management(fc, football_line_fields[1], "TEXT", field_length = 10)
        
    cursor = arcpy.da.InsertCursor(fc, football_line_fields)
    markers = [10, 20, 30, 40, 50, 60, 70, 80, 90]
    for marker in markers:
        line_1 = [(marker * 10, 533.3 / 2), (marker * 10, 0)]
        line_2 = [(marker * 10, 533.3 / 2), (marker * 10, 533.3)]
        if marker > 50: 
            cursor.insertRow([line_1, str(100 - marker)])
            cursor.insertRow([line_2, str(100 - marker)])
        else:
            cursor.insertRow([line_1, str(marker)])
            cursor.insertRow([line_2, str(marker)])

This is what it should look like: 

![Field](https://arcpy.files.wordpress.com/2016/02/superbowl_xlix-_field.jpg)

In [ ]:
# Drive chart showing results of every drive from Superbowl XLIX. 
# nflgame access drive data, arcpy creates drive chart feature class
# insert cursor adds the geometries and attribution to feature class

home = 'SEA' 
away = 'NE'
year = 2014
week = 5 
reg_post = 'POST'

print('Getting game data.')
game = nflgame.one(year, week, home, away, reg_post)

print('Getting game data.')
drives = get_game_drives(game)
drive_count = get_num_drives(drives)

def get_game_drives(game):
    # returns a list of drives
    return [drive for drive in game.drives]

def get_num_drives(drives):
    # returns the number of drives per game
    for drive in drives:
        drive_count = drive.drive_num
    
    return drive_count

drive_fields = ('SHAPE@', 'TEAM', 'DRIVE_NUM', 'START_POS', 'END_POS', 'DURATION', 'RESULT', 'DESCRIPTION')
# created feature class to insert drive data into 

# 'SHAPE@' is the shape of bar representing the drive
# 'TEAM' is the team that had possession of the football
# 'DRIVE_NUM' is the number of the drive
# 'START_POS' is the yardline where the drive started
# 'END_POS' is the yardline where the drive ended
# 'DURATION' is the length of time the drive took
# 'RESULT' is the outcome of the drive
# 'DESCRIPTION' is the NFL GameCenter description of the drive

In [ ]:
# Insert cursor for the drive chart feature class
# for every drive in the drives object, we created a drive chart polygon and populated the above attribute fields

print('Opening insert cursor')
cursor = arcpy.da.InsertCursor(os.path.join(output_gdb, output_fc), drive_fields)

drive_bar_height = 533.3 / drive_count
for drive in drives: 
    if drive.field_start:
        start_x, end_x = create_chart_polygon(drive, home, away)
        if start_x == end_x: 
            polygon == [
                (start_x, (drive_count - drive.drive_num) * drive_bar_height),
                (end_x + 0.1, (drive_count - drive.drive_num) * drive_bar_height),
                (end_x + 0.1, (drive_count - drive.drive_num) * drive_bar_height + (drive_bar_height - 1)),
                (start_x, (drive_count - drive.drive_num) * drive_bar_height + (drive_bar_height -1))]
        else:
            polygon == [
                (start_x, (drive_count - drive.drive_num) * drive_bar_height),
                (end_x, (drive_count - drive.drive_num) * drive_bar_height),
                (end_x, (drive_count - drive.drive_num) * drive_bar_height + (drive_bar_height - 1)),
                (start_x, (drive_count - drive.drive_num) * drive_bar_height + (drive_bar_height - 1))]
        
        cursor.insertRow([polygone, drive.team, drive.drive_num, str(drive.field_start), str(drive.field_end), str(drive.pos_time._dict_['minutes']) + ' min and ' + str(drive.pos_time._dict_['seconds']) + ' sec', drive.result, str(drive)])
    

In [ ]:
def create_chart_polygon(drive, home, away):
    # returns the start position (start_x) and end position (end_x) for each drive
    scale_by = 10 
    if drive.team == home:
        start_x = 50 + drive.field_start._dict_['offset']
        if drive.result == 'Touchdown':
            end_x = 100
        else:
            end_x = 50 + drive.field_end._dict_['offset']
            
        if drive.team == away:
            start_x = 50 - drive.field_start._dict_['offset']
            if drive.result == 'Touchdown':
                end_x = 0
            else:
                end_x = 50 - drive.field_start._dict_['offset']
        
        return scale_by * start_x, scale_by * end_x 

After creating the drive chart, we added it to the football field, symbolized drives by 'TEAM' field and added labels corresponding to the 'RESULT' field. 

![Field2](https://arcpy.files.wordpress.com/2016/02/superbowl_xlix-_drives.jpg)